## Supplemental Analysis of SNe-heated Gas Particles 
`-- Leo Lonzarich, 27 September 2021 // [Updated 28-9-21] --`

<i>Supplementing work of Akins H. in 'Justice_League_Code/Analysis/RamPressure/RamPressure.ipynb'</i>


### Table of Contents
- [SN-assisted ram pressure stripping](#rampressure)
- [Exit Angles](#exitangle)
- [Mass Across Halos](#mass)
- [Physical Distribution](#distribution)

In [ ]:
import numpy as np
import pylab
import pynbody
import pynbody.plot.sph as sph
import matplotlib.pyplot as plt

# importing custom analysis package developed by Akins H.
#from analysis import *

from importlib.machinery import SourceFileLoader
analysis = SourceFileLoader('analysis', '/home/lonzaric/astro_research/Justice_League_Code/Analysis\
/RamPressure/analysis.py').load_module()

In [ ]:
path = '/home/lonzaric/astro_research/ahdata/ram_pressure.hdf5'
with pd.HDFStore(path) as hdf:
    keys = np.array([h[1:] for h in hdf.keys()])

# listing dwarf galaxies of simulations that are viable for study (gaseous, satellite, etc).
print('Available keys:', *keys)

In [ ]:
key = 'h148_68'
data = pd.read_hdf(path, key=key)


data.head()

In [ ]:
import warnings
warnings.filterwarnings("ignore")

data_all = read_all_ram_pressure()

In [ ]:
# importing simulation 

h148 = pynbody.load('../h148.cosmo50PLK.3072g3HbwK1BH/snapshots_200bkgdens/h148.cosmo50PLK.3072g3HbwK1BH.004096')
h229 = pynbody.load('../h229.cosmo50PLK.3072gst5HbwK1BH/snapshots_200bkgdens/h229.cosmo50PLK.3072gst5HbwK1BH.004096')
h242 = pynbody.load('../h242.cosmo50PLK.3072gst5HbwK1BH/snapshots_200bkgdens/h242.cosmo50PLK.3072gst5HbwK1BH.004096')
h329 = pynbody.load('../h329.cosmo50PLK.3072gst5HbwK1BH/snapshots_200bkgdens/h329.cosmo50PLK.3072gst5HbwK1BH.004096')

In [ ]:
# halo numbers for halos with stars in them and fMhires > 0.9 [updated 09-08-21]

nums_h148 = [1, 2, 3, 5, 6, 9, 10, 11, 13, 14, 21, 24, 27, 28, 30, 32, 36, 37, 41, 45, 47, 48, 58, 61, 65, 68, 80, 81, 96, 105, 119, 127, 128, 136, 163, 212, 265, 278, 283, 329, 372, 377, 384, 386, 442, 491, 620, 678, 699, 711, 759, 914, 1004, 1024, 1201, 1217, 1451, 2905, 5039]
nums_h229 = [1, 2, 5, 7, 17, 20, 22, 23, 27, 29, 33, 52, 53, 55, 59, 61, 62, 73, 104, 113, 139, 212, 290, 549, 1047, 1374, 1483, 1558, 6045]
nums_h242 = [1, 10, 12, 24, 30, 34, 40, 41, 44, 48, 49, 71, 78, 80, 86, 165, 223, 439, 480, 1872, 2885, 6054, 9380, 10426, 12297]
nums_h329 = [1, 11, 31, 33, 40, 64, 103, 133, 137, 146, 185, 447, 729, 996, 1509]

## From RamPressure: 
_______________
_______________

### How does SN-heating affect ram pressure stripping? <a class="anchor" id="rampressure"></a>

In [ ]:
ratios_sne, N_e_sne, N_p_sne, E_prob_sne = np.array([]),np.array([]),np.array([]),np.array([])

for key in np.unique(data_all.key):
    # SNe-affected, currently or in last {t_thresh} Gyr
    t_thresh = 0.5 

    d = data_all[data_all.key==key]
    expelled = pd.read_hdf(f'/home/lonzaric/astro_research/ahdata/expelled_particles.hdf5', key=key)
    expelled = expelled[expelled.coolontime > expelled.time - t_thresh]
    particles = read_tracked_particles(key[:4],int(key[5:]))
    particles = particles[(particles.in_sat)&(particles.coolontime > particles.time - t_thresh)]

    expelled_grp = expelled.groupby(['time']).pid.count().reset_index()
    expelled_grp = expelled_grp.rename(columns={'pid':'N_e'})
    particles_grp = particles.groupby(['time']).pid.count().reset_index()
    particles_grp = particles_grp.rename(columns={'pid':'N_p'})

    d = pd.merge(d.sort_values('t'), expelled_grp.sort_values('time'), left_on='t', right_on='time', how='left')
    d = pd.merge(d.sort_values('t'), particles_grp.sort_values('time'), left_on='t', right_on='time', how='left')

    times = np.unique(d.time)
    ratios1 = np.array(d.ratio)
    N_e1 = np.array(d.N_e)[1:]
    N_p1 = np.array(d.N_p)[:-1]
    E_prob1 = N_e1/N_p1
    ratios1 = ratios1[1:]

    ratios_sne = np.append(ratios_sne,ratios1)
    N_e_sne = np.append(N_e_sne, N_e1)
    N_p_sne = np.append(N_p_sne, N_p1)
    E_prob_sne = np.append(E_prob_sne, E_prob1)

In [ ]:
cond_sne = ~np.isnan(E_prob_sne) & ~np.isinf(E_prob_sne)
ratios_sne = ratios_sne[cond_sne]
N_e_sne = N_e_sne[cond_sne]
N_p_sne = N_p_sne[cond_sne]
E_prob_sne = E_prob_sne[cond_sne]

bins = np.append(np.arange(-4,2,1),2.2)
bc = 0.5*(bins[1:] + bins[:-1])
E_prob_median_sne, E_prob_weighted_sne, E_prob_weighted_log_sne, std_sne = np.array([]),np.array([]),\
    np.array([]),np.array([])

for left_edge, right_edge in zip(bins[:-1],bins[1:]):
    cond = (np.log10(ratios_sne) >= left_edge) & (np.log10(ratios_sne) < right_edge)
    E_prob_weighted_sne = np.append(E_prob_weighted_sne,np.average(E_prob_sne[cond], weights=N_p_sne[cond]))
    E_prob_weighted_log_sne = np.append(E_prob_weighted_log_sne,np.average(E_prob_sne[cond], weights=\
        np.log10(N_p_sne)[cond]))
    E_prob_median_sne = np.append(E_prob_median_sne,np.median(E_prob_sne[cond]))
    std_sne = np.append(std_sne, np.std(E_prob_sne[cond]))

### Probability of expulsion by ram pressure for sne-heated gas

Recall that the pressure ratio used here is defined to be $\mathcal{P} \equiv P_{\rm ram}/P_{\rm rest}$. 

In [ ]:
fig, ax = plt.subplots(1,1,dpi=250, figsize=(3.2,2.5), constrained_layout=True)

ax.scatter(np.log10(ratios_sne), E_prob_sne, c='0.7', s=1)
ax.plot(bc, E_prob_weighted_sne, 'ko-', ms=4, label=r'Mean (weighted by $N_p(t_1)$)')
ax.plot(bc, E_prob_weighted_log_sne, 'ro-', ms=4, label=r'Mean (weighted by $\log_{10}(N_p(t_1))$)')
ax.plot(bc, E_prob_median_sne, 'bo-', ms=4, label='Median')

ax.legend(frameon=False, fontsize=7)
ax.set_xlabel(r'$\log_{10}(\mathcal{P})$')
ax.set_ylabel(r'Expulsion Probability $N_{e}(t_2)/N_{p}(t_1)$')
ax.set_xlim(-4, 3)
ax.set_ylim(-0.05, 1)
ax.tick_params(direction='in', which='both', top=True, right=True)
ax.xaxis.set_minor_locator(mpl.ticker.MultipleLocator(0.2))
ax.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(0.05))
ax.grid(alpha=0.1, zorder=-1000)
plt.show()

In [ ]:
 fig, ax = plt.subplots(1,1,dpi=200, figsize=(3,2.5), sharex=True)

ax.scatter(np.log10(ratios_sne), E_prob_sne, c='lightblue', s=1, alpha=0.5)
ax.fill_between(bc, E_prob_median_sne-std_sne, E_prob_median_sne+std_sne, color='r', ec=None, alpha=0.1)
ax.plot(bc, E_prob_median_sne, 'rD-', ms=4, label=r'Expulsion of recently SNe-affected gas')

ax.legend(frameon=False, fontsize=7)
ax.set_xlabel(r'$\log_{10}(\mathcal{P})$')
ax.set_ylabel(r'Expulsion Probability $N_{e}(t_2)/N_{p}(t_1)$')
ax.set_xlim(-4, 2.2)
ax.set_ylim(-0.05, 1)
ax.tick_params(direction='in', which='both', top=True, right=True)
ax.xaxis.set_minor_locator(mpl.ticker.MultipleLocator(0.2))
ax.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(0.05))
ax.grid(alpha=0.1, zorder=-1000)

plt.show()

## From ExitAngle: <a class="anchor" id="exitangle"></a>

____
____